In [1]:
import numpy as np
import cv2
import requests
import os, os.path

In [2]:
#Parameters
frames_DIR = 'C:\\Users\\NCTU\\Desktop\\WhoTakeWhat\\data\\Camera'
url = 'http://140.113.86.149:8896'

In [3]:
#Delete prior files in server
dr = requests.post(url + '/deletefiles')
print(dr.text)

remove success


In [4]:
#Update frames to server
from ipywidgets import IntProgress
from IPython.display import display

frames_files = [name for name in os.listdir(frames_DIR) if '.jpg' in name]

#init progress
progress = IntProgress()
progress.max = len(frames_files)
progress.description = '(Init)'
display(progress)

# # Simulating task execution
# for task in todo_tasks:
#     progress.value += 1
#     time.sleep(0.05)
#     progress.description = task
# progress.description = '(Done)'

images_count = len(frames_files)
                   
for count in range(images_count):
    files = {'file': open(frames_DIR + ('/%d.jpg' % count), 'rb')}
    r = requests.post(url + '/update', files = files)
    # Show progress
    progress.value += 1
    progress.description = ('%d.jpg to server: ' % count)
progress.description = '(Done)'
#     print('Update %d.jpg to server: ' % count, r.text)

IntProgress(value=0, description='(Init)', max=801)

In [5]:
#Get result from server
result = requests.get(url + '/result/track_all')
print(result.text)

{"taken": {"book": [476, 506], "bottle": [603, 641]}, "object_list": ["bottle", "book"]}


# Store all message

In [6]:
import json
import pandas as pd

original = pd.read_csv('C:/Users/NCTU/Desktop/WhoTakeWhat/data/Camera/original.csv', header=None)

decodejson =  json.loads(result.text)
print('Object list :', decodejson['object_list'])

takenOrNot_series = np.zeros((original.shape[0]))
what_series = ['n'] * original.shape[0]

object_taken_list = []
taken_dict = decodejson['taken']
for what, when in taken_dict.items():
    print(what, when)
    object_taken_list.append([what, when])
    takenOrNot_series[when[0]:when[1]+1] = 1
    for index, whatinser in enumerate(what_series):
        if index in range(when[0], when[1]+1):
            if whatinser == 'n':
                whatinser = what
            else:
                whatinser = whatinser + ',' + what
            what_series[index] = whatinser
        # what_series[when[0]:when[1]+1] = [what] * (when[1] - when[0]+1)
        

final_csv = pd.concat([pd.Series(takenOrNot_series, dtype='int16'),
           pd.Series(what_series),
           original],
          ignore_index=True, axis=1)

print(object_taken_list)
pd.DataFrame(object_taken_list).to_csv('C:/Users/NCTU/Desktop/WhoTakeWhat/data/result/Event_Detection/objects/taken_list.csv', header=None, index=None)
final_csv.to_csv('C:/Users/NCTU/Desktop/WhoTakeWhat/data/result/Event_Detection/objects/objects.csv', header=None, index=None)

Object list : ['bottle', 'book']
book [476, 506]
bottle [603, 641]
[['book', [476, 506]], ['bottle', [603, 641]]]
